<a href="https://colab.research.google.com/github/JosephBless/DL/blob/main/W%26B_Artifacts_API_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W&B Artifacts API Demo

In this notebook, we'll show you how to use W&B Artifacts to keep track of dataset versions in Weights & Biases.

### How it works
 Using our Artifacts API, you can log artifacts as outputs of W&B runs, or use artifacts as input to runs.
 

<img src="https://i.imgur.com/Pw8luzD.png" width="500">

Since a run can use another run’s output artifact as input, artifacts and runs together form a directed graph. You don’t need to define pipelines ahead of time. Just use and log artifacts, and we’ll stitch everything together.

## 1. Install Weights & Biases

In [ ]:
# Install Weights & Biases
!pip install wandb -qqq

import os
import random
import wandb

## 2. Create an Artifact

Call `wandb.init` to initialize a new run. Use a run to track any script in your pipeline— anything from training and evaluation to scraping and preprocessing data. Specify what type of run it is in the **job_type**, and you'll be able to filter and group based on **job_type** in the web interface.

`wandb.Artifact()`
- **name**: The name of this dataset, model, or other set of files. The files inside can change, but the name stays the same across versions.
- **type**: This helps you group together top-level artifacts, like "dataset" and "model".
- **metadata**: This dictionary lets you track class distributions, UUIDs, and any important details you want to save and search over later.

In [ ]:
# This will create a new run in the W&B database.
# Init starts tracking stdout/stderr and system metrics automatically.
run = wandb.init(project='artifacts-demo', job_type='producer')

# Create an Artifact. Give it a name and a type. Type is used for
# organizational purposes and should typically be "dataset" or "model".
artifact = wandb.Artifact('mountain-view-scenes', type='dataset', metadata={})

# Store a new file in the artifact
with artifact.new_file('overview.txt') as f:
    f.write('This is my first artifact!\n')

# Save the artifact to W&B. It will be tracked as output of the current run
# and appended to the Artifact Sequence called 'hello artifacts dataset'.
run.log_artifact(artifact)

Open [wandb.ai](https://app.wandb.ai/home) and click on your latest run to see the artifact tab appear on the left sidebar.

## 3. Use that Artifact
Next, let's use that artifact as input to another run.

In [ ]:
# Start a new run
run = wandb.init(project='artifacts-demo', job_type='consumer')

# We'll use the artifact we created in the previous run as input to this run.
# This will fetch the latest entry in the 'hello artifacts dataset' Artifact Sequence
artifact = run.use_artifact('mountain-view-scenes:latest')

# Download all of the files contained in the artifact.
artifact_dir = artifact.download()

# Let's take a look at the downloaded files contents
print(open(os.path.join(artifact_dir, 'overview.txt')).read())

Let's log an output artifact too— in this example it's just a fake model file.

In [ ]:
artifact = wandb.Artifact('run-%s-model' % run.id, type='model')

# This time we'll use artifact.add_file, to add a file that already exists.
f = open('mymodel.txt', 'w')
f.write('This is a really awesome trained model: %s' % random.random())
f.close()

artifact.add_file('mymodel.txt')
run.log_artifact(artifact)

# end the current run
run.finish()

Now you can navigate to your project page (linked above), and then click on the artifacts tab, to dig into all the artifacts you've created so far.

If you click through to an artifact, and then click on the "Graph" tab, you'll see a visualization that shows how your artifacts and runs are related to each other.

## Documentation

For more details, [see the docs →](https://docs.wandb.com/artifacts)
- Storing directories in artifacts
- Referring to external data using references
- Automatic file and artifact deduplication
- Best practices for dataset versioning and model management